In [ ]:
import os
import numpy as np
import cv2
import tqdm
import seaborn as sns
import matplotlib.pylab as plt
from keras.utils import to_categorical
import pandas as pd
from sklearn.utils import class_weight
from sklearn.metrics import accuracy_score, f1_score, classification_report
import gc
%matplotlib inline


Using TensorFlow backend.


In [17]:
main_path = '/kaggle/'
main_path= '/Users/dasm/projects/Gomrade/'
root = main_path + 'input/mygodataset/'
root = main_path + 'data/'



In [3]:
X_MARGIN = 16
Y_MARGIN = 16

data = pd.read_csv(root + 'data.csv')
columns = [str(i) for i in range(X_MARGIN*Y_MARGIN*2*2)]

sources = data['source']
examples = data['example']
labels = data['label']
image_inds = data['image_ind'].values.astype(np.uint16)
data = data[columns].values.astype(np.uint8) #[:11000, :]

labels = labels.values #[:11000]
sources = sources.values #[:11000]
examples = examples.values #[:11000]
labels = to_categorical(labels)


img_wh = X_MARGIN + Y_MARGIN
num_channels = 1
input_shape = (img_wh, img_wh, num_channels)
# input_shape = (img_wh*img_wh,)
num_classes = labels.shape[1]
print("Num classes: {}".format(num_classes))

/Users/dasm/venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (1028) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Num classes: 3


In [43]:
# input_shape = (img_wh*img_wh,)
input_shape = (img_wh, img_wh, num_channels)


In [45]:
# Importing the required Keras modules containing model and layers
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
from keras.optimizers import Adam
import tensorflow as tf

# def base_model(img_w, img_h, input_shape, num_classes):

#     model = Sequential()
#     model.add(Conv2D(img_w, (3, 3), padding='same', activation='relu', input_shape=input_shape))
#     model.add(Conv2D(img_w,(3, 3), activation='relu'))
#     model.add(MaxPooling2D(pool_size=(2, 2)))
#     model.add(Dropout(0.25))

#     model.add(Conv2D(img_w*2, (3, 3), padding='same', activation='relu'))
#     model.add(Conv2D(img_w*2, (3,3), activation='relu'))
#     model.add(MaxPooling2D(pool_size=(2, 2)))
#     model.add(Dropout(0.25))

#     model.add(Flatten())
#     model.add(Dense(512, activation='relu'))
#     model.add(Dropout(0.5))
#     model.add(Dense(num_classes, activation='softmax'))
#     optimizer = Adam(lr=0.001)
#     return model, optimizer


def base_model(img_w, img_h, input_shape, num_classes):
    model = Sequential()
    model.add(Conv2D(int(img_w), kernel_size=(3,3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(int(img_w*2), kernel_size=(3,3), activation='relu'))
    model.add(Dropout(0.4))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten()) # Flattening the 2D arrays for fully connected layers
    model.add(Dense(img_w*2, activation=tf.nn.relu))
    model.add(Dropout(0.4))
    model.add(Dense(num_classes, activation=tf.nn.softmax))
    optimizer = Adam(lr=0.001)
    return model, optimizer

# def base_model(img_w, img_h, input_shape, num_classes):
#     model = Sequential()
#     model.add(Dense(img_w*4, activation=tf.nn.relu, input_shape=input_shape))
#     model.add(Dropout(0.4))
#     model.add(Dense(num_classes, activation=tf.nn.softmax))
#     optimizer = Adam(lr=0.001)
#     return model, optimizer

model, optimizer = base_model(img_wh, img_wh, input_shape, num_classes)
model.summary()


Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 30, 30, 32)        320       
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 13, 13, 64)        18496     
_________________________________________________________________
dropout_33 (Dropout)         (None, 13, 13, 64)        0         
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 6, 6, 64)          0         
_________________________________________________________________
flatten_11 (Flatten)         (None, 2304)              0         
_________________________________________________________________
dense_45 (Dense)             (None, 64)              

In [46]:
import time
def evaluate_time(model, x_test, y_test):
    ex = x_test[:361]
    start = time.time()
    model.predict(ex)
    model.predict(ex)
    model.predict(ex)
    model.predict(ex)
    model.predict(ex)
    print("Image inference: {}".format((time.time() - start)/5))
    
    ex = x_test[:361*5]

    start = time.time()
    model.predict(ex)
    print("Image inference: {}".format((time.time() - start)/5))


In [47]:
#  import sys
# # These are the usual ipython objects, including this one you are creating
# ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

# # Get a sorted list of the objects and their sizes
# sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)


In [ ]:
min_limit = 361*10
accs = []
accepted_sources = []
all_y_test = []
all_test_predictions = []
all_y_train = []
all_train_predictions = []
all_correct = 0

for source in set(sources):
    if source == "Potorti'-DeLazzari" or source == "20_03_29_16_33_37" or source == "DeLazzari-Greenberg":
        pass
    else:
        # 20_03_29_16_33_37 # DeLazzari-Greenberg
        # 0.9459174275398254,0.9825117588043213,0.9658606052398682
        continue
    accepted_sources.append(source)
    start = time.time()
    print('\n\n')
    print('='*100)
    print('{}'.format(source))

    test_ind = np.where(sources == source)[0]
    train_ind = np.delete(np.arange(len(data)), test_ind)
    if len(test_ind) < min_limit:
        print('skipping {}'.format(source))
        continue
    
    ex_test = examples[test_ind]
    src_test = sources[test_ind]
    imind_test = image_inds[test_ind]
    
    x_train = data[train_ind]/255
    x_test = data[test_ind]/255
    y_train = labels[train_ind]
    y_test = labels[test_ind]
    curr_inds = image_inds[test_ind]
    
    inds = np.where(np.argmax(y_train, axis=1) >0)[0]
#     Reshaping the array to 4-dims so that it can work with the Keras API
    x_train = x_train.reshape(x_train.shape[0], img_wh, img_wh, num_channels)
    x_test = x_test.reshape(x_test.shape[0], img_wh, img_wh, num_channels)

    # Augmentation
#     x_train = x_train.reshape(x_train.shape[0], img_wh, img_wh, num_channels)
    x_train = np.vstack((x_train, x_train[inds, : ,::-1, :]))
    y_train = np.vstack((y_train, y_train[inds]))
                     
#     x_train = x_train.reshape(x_train.shape[0], img_wh * img_wh)
#     x_test = x_test.reshape(x_test.shape[0], img_wh * img_wh)
    
    # Making sure that the values are float so that we can get decimal points after division
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    print('x_train shape:', x_train.shape)
    print('Number of images in x_train', x_train.shape[0])
    print('Number of images in x_test', x_test.shape[0])

    sns.countplot(np.argmax(y_train, axis=1))

    class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(np.argmax(y_train, axis=1)),
                                                 np.argmax(y_train, axis=1))
    model, optimizer = base_model(img_wh, img_wh, input_shape, num_classes)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
#     plot_losses = TrainingPlot(source)
    history = model.fit(x_train, y_train, batch_size=128, epochs=1, verbose=False, validation_data=(x_test, y_test), class_weight=class_weights)
    
    loss, accuracy  = model.evaluate(x_test, y_test, verbose=False)
    accs.append(accuracy)
    
    if accuracy == 1.0:
        all_correct += 1

    plt.figure()
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['training', 'validation'], loc='best')
    plt.show()

    print(f'Test loss: {loss:.3}')
    print(f'Test accuracy: {accuracy:.3}')
    
    evaluate_time(model, x_test, y_test)
    
    predictions  = model.predict(x_test, verbose=False)
    print(classification_report(np.argmax(y_test, axis=1), np.argmax(predictions, axis=1), digits=5))
    
    all_y_test.extend(np.argmax(y_test, axis=1).flatten().tolist())
    all_test_predictions.extend(np.argmax(predictions, axis=1).flatten().tolist())

    limit = 10
    ea_ind = 0
    for i, (ref, pred) in enumerate(zip(np.argmax(y_test, axis=1), np.argmax(predictions, axis=1))):
        if ref != pred:
            plt.figure(figsize=(4,4))
            plt.imshow(x_test[i].reshape(X_MARGIN*2, Y_MARGIN*2))
            plt.title(f'ref: {ref} \npre: {pred}\nsrc: {src_test[i]}\nex: {ex_test[i]}\nind {imind_test[i]}', family='monospace')
            plt.savefig(f'{main_path}/working/ref_{ref}_pre_{pred}_src_{src_test[i]}_ex_{ex_test[i]}_ind_{imind_test[i]}.jpg')
            ea_ind += 1
        if ea_ind < limit:
            plt.show()
        else:
            plt.close()
                
    predictions  = model.predict(x_train, verbose=False)
    all_y_train.extend(np.argmax(y_train, axis=1).flatten().tolist())
    all_train_predictions.extend(np.argmax(predictions, axis=1).flatten().tolist())
        
    print(time.time() - start)


print(f'Full src acc: {all_correct/len(accepted_sources)*100}')
print(f'Ave position accuracy: {np.mean(accs):.5}')
print(f'Position std: {np.std(accs):.5}')
print('Test:')
print(classification_report(all_y_test, all_test_predictions, digits=5))
print('Train:')
print(classification_report(all_y_train, all_train_predictions, digits=5))



In [ ]:
model.reset_metrics()
model.save('go_model.h5')


In [196]:
'''
373.66814279556274
0.9944631088347662
0.013440826207001617
              precision    recall  f1-score   support

           0    0.99823   0.99548   0.99685    296319
           1    0.98055   0.99471   0.98758     31775
           2    0.97602   0.98710   0.98153     31462

    accuracy                        0.99468    359556
   macro avg    0.98493   0.99243   0.98865    359556
weighted avg    0.99472   0.99468   0.99469    359556

              precision    recall  f1-score   support

           0    0.99971   0.99974   0.99972   6175440
           1    0.99942   0.99933   0.99938   1317410
           2    0.99885   0.99879   0.99882   1305268

    accuracy                        0.99954   8798118
   macro avg    0.99933   0.99929   0.99931   8798118
weighted avg    0.99954   0.99954   0.99954   8798118

'''

In [51]:
for acc, src in zip(accs, accepted_sources):
    print(acc, src)

0.9852484464645386 20_03_29_16_33_37
0.9761173725128174 DeLazzari-Greenberg
0.9713758230209351 Potorti'-DeLazzari


In [ ]:
0.9972299337387085 20_03_29_16_33_37
0.9798607230186462 DeLazzari-Greenberg
0.9231631755828857 Potorti'-DeLazzari

In [ ]:
0.9965290427207947 20_03_29_16_33_37
0.9826308488845825 DeLazzari-Greenberg
0.9401134252548218 Potorti'-DeLazzari

In [ ]:
0.997430145740509 20_03_29_16_33_37
0.9577749371528625 DeLazzari-Greenberg
0.9872708320617676 Potorti'-DeLazzari